In [ ]:
# Tests of zfturbo script "Santander Battle"

In [1]:
import datetime
import os
from collections import defaultdict
import operator

In [2]:
DATA_PATH = "../data"

In [3]:
f = open(os.path.join(DATA_PATH, "train_ver2_201601-201605.csv"), 'r')
first_line = f.readline().strip()
# Remove " symbols in the header. Header is like : "a","b","c"...
first_line = first_line.replace("\"", "")
# map names <-> target labels
map_names = first_line.split(",")[24:]
print "MAP NAMES : ", map_names

MAP NAMES :  ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


In [4]:
# Normal variables
customer = dict()
best = defaultdict(lambda: defaultdict(int))
overallbest = defaultdict(int)

# Validation variables
customer_valid = dict()
best_valid = defaultdict(lambda: defaultdict(int))
overallbest_valid = defaultdict(int)

valid_part = []

In [5]:
# Calc counts
total = 0
while 1:
    line = f.readline()[:-1]
    total += 1

    if line == '':
        break

    # Parse line : 
    if "\"" in line:
        tmp1 = line.split("\"")
        arr = tmp1[0][:-1].split(",") + [tmp1[1]] + tmp1[2][1:].split(',')
    else:
        arr = line.split(",")
    arr = [a.strip() for a in arr]

    # Normal part
    add_data_to_main_arrays(arr, best, overallbest, customer)

    
    # Valid part
    if arr[0] != '2016-05-28':
        add_data_to_main_arrays(arr, best_valid, overallbest_valid, customer_valid)
    else:
        valid_part.append(arr)

    if total % 1000000 == 0:
        print('Process {} lines ...'.format(total))
        #break

NameError: name 'add_data_to_main_arrays' is not defined

In [14]:
f.close()

In [15]:
print('Sort best arrays...')
print('Hashes num: ', len(best))
print('Hashes valid num: ', len(best_valid))
print('Valid part: ', len(valid_part))

Sort best arrays...
('Hashes num: ', 43536)
('Hashes valid num: ', 42684)
('Valid part: ', 931453)


In [17]:
# Normal
best, overallbest = sort_main_arrays(best, overallbest)

# Valid
best_valid, overallbest_valid = sort_main_arrays(best_valid, overallbest_valid)

In [ ]:
map7 = 0.0
print('Validation...')
count = -1
for i, arr1 in enumerate(valid_part):
    
    predicted = get_predictions(arr1, best_valid, overallbest_valid, customer_valid)
    real = get_real_values(arr1, customer_valid)
    score = apk(real, predicted)
    #print "-- i : ", i, arr1[1], " score : ", score, " | predicted : ", predicted, ", real : ", real
    map7 += score
    
    count-= 1
    if count == 0:
        break

if len(valid_part) > 0:
    map7 /= len(valid_part)
print('Predicted score: {}'.format(map7))

In [19]:
print('Generate submission...')
sub_file = os.path.join('../results','zft_0711_02.csv')
out = open(sub_file, "w")
f = open(DATA_PATH+"/test_ver2.csv", "r")
f.readline()
total = 0
out.write("ncodpers,added_products\n")

while 1:
    line = f.readline()[:-1]
    total += 1

    if line == '':
        break

    tmp1 = line.split("\"")
    arr = tmp1[0][:-1].split(",") + [tmp1[1]] + tmp1[2][1:].split(',')
    arr = [a.strip() for a in arr]
    ncodpers = arr[1]
    out.write(ncodpers + ',')

    predicted = get_predictions(arr, best, overallbest, customer)

    for p in predicted:
        out.write(map_names[p] + ' ')

    if total % 1000000 == 0:
        print('Read {} lines ...'.format(total))
        # break

    out.write("\n")

print('Total cases:', str(total))
out.close()
f.close()

Generate submission...
('Total cases:', '929616')


In [6]:
def get_predictions(arr1, best, overallbest, customer):

    predicted = []

    hash1 = get_hash(arr1, 0)
    hash2 = get_hash(arr1, 1)
    ncodpers = arr1[1]

    # hash 1
    if len(predicted) < 7:
        if hash1 in best:
            for a in best[hash1]:
                # If user is not new
                if ncodpers in customer:
                    if customer[ncodpers][a[0]] == '1':
                        continue
                if a[0] not in predicted:
                    predicted.append(a[0])
                    if len(predicted) == 7:
                        break

    # hash 2
    if len(predicted) < 7:
        if hash2 in best:
            for a in best[hash2]:
                # If user is not new
                if ncodpers in customer:
                    if customer[ncodpers][a[0]] == '1':
                        continue
                if a[0] not in predicted:
                    predicted.append(a[0])
                    if len(predicted) == 7:
                        break

    # overall
    if len(predicted) < 7:
        for a in overallbest:
            # If user is not new
            if ncodpers in customer:
                if customer[ncodpers][a[0]] == '1':
                    continue
            if a[0] not in predicted:
                predicted.append(a[0])
                if len(predicted) == 7:
                    break

    return predicted

In [7]:
def get_real_values(arr1, customer):
    real = []
    ncodpers = arr1[1]
    arr2 = arr1[24:]

    for i in range(len(arr2)):
        if arr2[i] == '1':
            if ncodpers in customer:
                if customer[ncodpers][i] == '0':
                    real.append(i)
            else:
                real.append(i)
    return real

In [8]:
def apk(actual, predicted, k=7):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [9]:
def get_hash(arr, type = 0):
    (fecha_dato, ncodpers, ind_empleado,
    pais_residencia, sexo, age,
    fecha_alta, ind_nuevo, antiguedad,
    indrel, ult_fec_cli_1t, indrel_1mes,
    tiprel_1mes, indresi, indext,
    conyuemp, canal_entrada, indfall,
    tipodom, cod_prov, nomprov,
    ind_actividad_cliente, renta, segmento) = arr[:24]

    if type == 0:
        return (pais_residencia, sexo, age, ind_nuevo, segmento, ind_empleado, ind_actividad_cliente, indresi, nomprov)
    else:
        return (pais_residencia, sexo, age, ind_nuevo, segmento, ind_empleado, ind_actividad_cliente, indresi)

hash : client description (place of residence, gender, age, etc)

best : map between client description and stat of popular products

overallbest : stats of popular products

customer : latest command

In [10]:
def sort_main_arrays(best, overallbest):
    out = dict()
    for b in best:
        arr = best[b]
        srtd = sorted(arr.items(), key=operator.itemgetter(1), reverse=True)
        out[b] = srtd
    best = out
    overallbest = sorted(overallbest.items(), key=operator.itemgetter(1), reverse=True)
    return best, overallbest

In [11]:
def add_data_to_main_arrays(arr, best, overallbest, customer):
    ncodpers = arr[1]
    #print "CLIENT : ", ncodpers
    hash1 = get_hash(arr, 0)
    #print "HASH 1 : ", hash1
    hash2 = get_hash(arr, 1)
    #print "HASH 2 : ", hash2
    part = arr[24:]
    for i in range(24):
        if part[i] == '1':
            if ncodpers in customer:
                if customer[ncodpers][i] == '0':
                    best[hash1][i] += 1
                    best[hash2][i] += 1
                    overallbest[i] += 1
            else:
                best[hash1][i] += 1
                best[hash2][i] += 1
                overallbest[i] += 1
    customer[ncodpers] = part

In [12]:
def verbose():
    print "\nBEST : ", best.items()
    print "\nOVERALLBEST : ", overallbest.items()
    print "\nCUSTOMER : ",customer.items()

In [ ]:
verbose()

## Compare csv files

In [15]:
file1 = '../results/zft_0711_02.csv'
file2 = '../results/submission_2016-11-16-13-31.csv'
#file2 = '../results/submission_2016-11-16-14-06.csv'

In [9]:
import numpy as np
import pandas as pd

In [10]:
df_1 = pd.read_csv(file1)

In [16]:
df_2 = pd.read_csv(file2)

In [12]:
df_1.head()

,ncodpers,added_products
0,15889,ind_ecue_fin_ult1 ind_fond_fin_ult1 ind_recibo...
1,1170544,ind_ctop_fin_ult1 ind_ctpp_fin_ult1 ind_ecue_f...
2,1170545,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_...
3,1170547,ind_cno_fin_ult1 ind_recibo_ult1 ind_ecue_fin_...
4,1170548,ind_recibo_ult1 ind_cno_fin_ult1 ind_ecue_fin_...


In [17]:
df_2.head()

,ncodpers,added_products
0,15889,ind_cco_fin_ult1 ind_dela_fin_ult1 ind_ecue_fi...
1,1170544,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_ecue_fi...
2,1170545,ind_cno_fin_ult1 ind_nom_pens_ult1 ind_recibo_...
3,1170547,ind_cno_fin_ult1 ind_recibo_ult1 ind_nom_pens_...
4,1170548,ind_cno_fin_ult1 ind_recibo_ult1 ind_nom_pens_...


In [14]:
print df_1.loc[0].values
print df_2.loc[0].values

[15889
 'ind_ecue_fin_ult1 ind_fond_fin_ult1 ind_recibo_ult1 ind_ctop_fin_ult1 ind_cno_fin_ult1 ind_nom_pens_ult1 ind_nomina_ult1 ']
[15889
 'ind_cco_fin_ult1 ind_dela_fin_ult1 ind_recibo_ult1 ind_ecue_fin_ult1 ind_ctop_fin_ult1 ind_fond_fin_ult1 ind_cno_fin_ult1 ']
